# FreshHarvest Model Experiments

This notebook contains experiments with different CNN architectures for fruit freshness classification.
We'll compare Basic CNN, Improved CNN with ResNet blocks, and Lightweight CNN architectures.

In [ ]:
import os
import sys
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from pathlib import Path
import yaml
import warnings
warnings.filterwarnings('ignore')

# Add src to path
sys.path.append('../src')

import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint, ReduceLROnPlateau

# Import custom modules
from cvProject_FreshHarvest.utils.common import read_yaml, setup_logging
from cvProject_FreshHarvest.models.cnn_models import FreshHarvestCNN

# Setup
setup_logging()
plt.style.use('seaborn-v0_8')
sns.set_palette("husl")

print(f"TensorFlow version: {tf.__version__}")
print(f"GPU available: {tf.config.list_physical_devices('GPU')}")

## 1. Load Configuration and Data

In [ ]:
# Load configuration
config = read_yaml('../config/config.yaml')
print("Configuration loaded:")
print(f"- Image size: {config['data']['image_size']}")
print(f"- Number of classes: {config['data']['num_classes']}")
print(f"- Batch size: {config['training']['batch_size']}")
print(f"- Learning rate: {config['training']['learning_rate']}")

In [ ]:
# Create data generators
def create_data_generators(batch_size=32):
    """Create training and validation data generators."""
    
    # Training data generator with augmentation
    train_datagen = ImageDataGenerator(
        rescale=1./255,
        rotation_range=20,
        width_shift_range=0.1,
        height_shift_range=0.1,
        horizontal_flip=True,
        zoom_range=0.1,
        brightness_range=[0.8, 1.2],
        fill_mode='nearest'
    )
    
    # Validation data generator (no augmentation)
    val_datagen = ImageDataGenerator(rescale=1./255)
    
    # Create generators
    train_generator = train_datagen.flow_from_directory(
        '../data/processed/train',
        target_size=tuple(config['data']['image_size']),
        batch_size=batch_size,
        class_mode='categorical',
        shuffle=True
    )
    
    val_generator = val_datagen.flow_from_directory(
        '../data/processed/val',
        target_size=tuple(config['data']['image_size']),
        batch_size=batch_size,
        class_mode='categorical',
        shuffle=False
    )
    
    return train_generator, val_generator

# Create generators
train_gen, val_gen = create_data_generators()
print(f"Training samples: {train_gen.samples}")
print(f"Validation samples: {val_gen.samples}")
print(f"Classes: {list(train_gen.class_indices.keys())}")

## 2. Model Architecture Comparison

In [ ]:
# Initialize CNN model builder
cnn_builder = FreshHarvestCNN('../config/config.yaml')

# Create different model architectures
models = {
    'Basic CNN': cnn_builder.create_basic_cnn(),
    'Improved CNN': cnn_builder.create_improved_cnn(),
    'Lightweight CNN': cnn_builder.create_lightweight_cnn()
}

# Display model summaries
for name, model in models.items():
    print(f"\n{'='*50}")
    print(f"Model: {name}")
    print(f"{'='*50}")
    print(f"Total parameters: {model.count_params():,}")
    print(f"Trainable parameters: {sum([tf.keras.backend.count_params(w) for w in model.trainable_weights]):,}")
    
    # Show model architecture
    model.summary()

## 3. Model Training Experiments

In [ ]:
def train_model_experiment(model, model_name, epochs=10):
    """Train a model and return training history."""
    
    # Compile model
    model.compile(
        optimizer=keras.optimizers.Adam(learning_rate=config['training']['learning_rate']),
        loss='categorical_crossentropy',
        metrics=['accuracy', 'precision', 'recall']
    )
    
    # Create callbacks
    callbacks = [
        EarlyStopping(
            monitor='val_loss',
            patience=5,
            restore_best_weights=True,
            verbose=1
        ),
        ReduceLROnPlateau(
            monitor='val_loss',
            factor=0.5,
            patience=3,
            min_lr=1e-7,
            verbose=1
        )
    ]
    
    print(f"\nTraining {model_name}...")
    
    # Train model
    history = model.fit(
        train_gen,
        epochs=epochs,
        validation_data=val_gen,
        callbacks=callbacks,
        verbose=1,
        steps_per_epoch=min(50, train_gen.samples // config['training']['batch_size']),
        validation_steps=min(20, val_gen.samples // config['training']['batch_size'])
    )
    
    return history

# Train all models (short training for comparison)
training_histories = {}

for name, model in models.items():
    print(f"\n{'='*60}")
    print(f"Training {name}")
    print(f"{'='*60}")
    
    try:
        history = train_model_experiment(model, name, epochs=5)
        training_histories[name] = history
        print(f"✅ {name} training completed successfully!")
    except Exception as e:
        print(f"❌ {name} training failed: {e}")
        training_histories[name] = None

## 4. Training Results Visualization

In [ ]:
# Plot training histories
def plot_training_comparison(histories):
    """Plot training metrics comparison across models."""
    
    fig, axes = plt.subplots(2, 2, figsize=(15, 10))
    
    metrics = ['accuracy', 'loss', 'precision', 'recall']
    titles = ['Training Accuracy', 'Training Loss', 'Training Precision', 'Training Recall']
    
    for idx, (metric, title) in enumerate(zip(metrics, titles)):
        ax = axes[idx // 2, idx % 2]
        
        for name, history in histories.items():
            if history is not None:
                if metric in history.history:
                    ax.plot(history.history[metric], label=f'{name} - Train', marker='o')
                if f'val_{metric}' in history.history:
                    ax.plot(history.history[f'val_{metric}'], label=f'{name} - Val', marker='s', linestyle='--')
        
        ax.set_title(title)
        ax.set_xlabel('Epoch')
        ax.set_ylabel(metric.capitalize())
        ax.legend()
        ax.grid(True, alpha=0.3)
    
    plt.tight_layout()
    plt.show()

# Plot comparison if we have training histories
if any(h is not None for h in training_histories.values()):
    plot_training_comparison(training_histories)
else:
    print("No successful training histories to plot.")

## 5. Model Performance Summary

In [ ]:
# Create performance summary
def create_performance_summary(models, histories):
    """Create a summary table of model performance."""
    
    summary_data = []
    
    for name, model in models.items():
        history = histories.get(name)
        
        # Model complexity metrics
        total_params = model.count_params()
        trainable_params = sum([tf.keras.backend.count_params(w) for w in model.trainable_weights])
        
        # Performance metrics (if training was successful)
        if history is not None:
            final_train_acc = history.history['accuracy'][-1] if 'accuracy' in history.history else 0
            final_val_acc = history.history['val_accuracy'][-1] if 'val_accuracy' in history.history else 0
            final_train_loss = history.history['loss'][-1] if 'loss' in history.history else float('inf')
            final_val_loss = history.history['val_loss'][-1] if 'val_loss' in history.history else float('inf')
            best_val_acc = max(history.history['val_accuracy']) if 'val_accuracy' in history.history else 0
        else:
            final_train_acc = final_val_acc = final_train_loss = final_val_loss = best_val_acc = 'N/A'
        
        summary_data.append({
            'Model': name,
            'Total Parameters': f'{total_params:,}',
            'Trainable Parameters': f'{trainable_params:,}',
            'Final Train Accuracy': f'{final_train_acc:.4f}' if isinstance(final_train_acc, float) else final_train_acc,
            'Final Val Accuracy': f'{final_val_acc:.4f}' if isinstance(final_val_acc, float) else final_val_acc,
            'Best Val Accuracy': f'{best_val_acc:.4f}' if isinstance(best_val_acc, float) else best_val_acc,
            'Final Train Loss': f'{final_train_loss:.4f}' if isinstance(final_train_loss, float) else final_train_loss,
            'Final Val Loss': f'{final_val_loss:.4f}' if isinstance(final_val_loss, float) else final_val_loss
        })
    
    return pd.DataFrame(summary_data)

# Create and display summary
summary_df = create_performance_summary(models, training_histories)
print("\nModel Performance Summary:")
print("=" * 100)
print(summary_df.to_string(index=False))

# Save summary to CSV
summary_df.to_csv('../outputs/model_comparison_summary.csv', index=False)
print("\n✅ Summary saved to outputs/model_comparison_summary.csv")

## 6. Model Architecture Visualization

In [ ]:
# Visualize model architectures
def visualize_model_architecture(model, model_name):
    """Create a visualization of the model architecture."""
    
    try:
        # Create model plot
        tf.keras.utils.plot_model(
            model,
            to_file=f'../outputs/{model_name.lower().replace(" ", "_")}_architecture.png',
            show_shapes=True,
            show_layer_names=True,
            rankdir='TB',
            expand_nested=True,
            dpi=150
        )
        print(f"✅ {model_name} architecture saved to outputs/")
    except Exception as e:
        print(f"❌ Failed to save {model_name} architecture: {e}")

# Create architecture visualizations
print("Creating model architecture visualizations...")
for name, model in models.items():
    visualize_model_architecture(model, name)

## 7. Conclusions and Recommendations

In [ ]:
# Analysis and recommendations
print("\n" + "="*80)
print("MODEL EXPERIMENT CONCLUSIONS")
print("="*80)

print("\n🔍 ARCHITECTURE ANALYSIS:")
print("-" * 40)

for name, model in models.items():
    params = model.count_params()
    print(f"\n{name}:")
    print(f"  • Parameters: {params:,}")
    print(f"  • Model size: ~{params * 4 / (1024*1024):.2f} MB (float32)")
    
    if 'Basic' in name:
        print(f"  • Characteristics: Simple CNN, good baseline")
    elif 'Improved' in name:
        print(f"  • Characteristics: ResNet blocks, better feature learning")
    elif 'Lightweight' in name:
        print(f"  • Characteristics: Separable convolutions, mobile-friendly")

print("\n📊 PERFORMANCE INSIGHTS:")
print("-" * 40)

# Find best performing model
best_model = None
best_val_acc = 0

for name, history in training_histories.items():
    if history is not None and 'val_accuracy' in history.history:
        max_val_acc = max(history.history['val_accuracy'])
        if max_val_acc > best_val_acc:
            best_val_acc = max_val_acc
            best_model = name

if best_model:
    print(f"\n🏆 Best performing model: {best_model}")
    print(f"   Best validation accuracy: {best_val_acc:.4f}")
else:
    print("\n⚠️  No successful training runs to compare")

print("\n💡 RECOMMENDATIONS:")
print("-" * 40)
print("1. For production deployment: Use Lightweight CNN for mobile/edge devices")
print("2. For maximum accuracy: Use Improved CNN with longer training")
print("3. For quick prototyping: Use Basic CNN as baseline")
print("4. Consider ensemble methods combining multiple architectures")
print("5. Implement transfer learning from pre-trained models for better performance")

print("\n🔄 NEXT STEPS:")
print("-" * 40)
print("1. Run hyperparameter tuning on the best performing architecture")
print("2. Implement data augmentation strategies")
print("3. Try transfer learning with pre-trained models")
print("4. Evaluate models on test set for final performance metrics")
print("5. Optimize models for deployment (quantization, pruning)")